In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv("clean_fifa_worldcup_matches.csv").dropna()
team_encoder = LabelEncoder()
df['HomeTeamEncoded'] = team_encoder.fit_transform(df['HomeTeam'])
df['AwayTeamEncoded'] = df['AwayTeam'].apply(lambda x: team_encoder.transform([x])[0] if x in team_encoder.classes_ else -1)
X = df[['HomeTeamEncoded', 'AwayTeamEncoded']]
y_home = df['HomeGoals']
y_away = df['AwayGoals']
X_train, X_test, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.2, random_state=42)
X_train, X_test, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.2, random_state=42)

In [3]:
home_model = RandomForestRegressor(n_estimators=100, random_state=42)
away_model = RandomForestRegressor(n_estimators=100, random_state=42)

home_model.fit(X_train, y_train_home)
away_model.fit(X_train, y_train_away)


RandomForestRegressor(random_state=42)

In [4]:
fixtures = pd.read_csv("clean_fifa_worldcup_fixture.csv")
fixtures.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam'}, inplace=True)
fixtures['HomeTeamEncoded'] = fixtures['HomeTeam'].apply(lambda x: team_encoder.transform([x])[0] if x in team_encoder.classes_ else -1)
fixtures['AwayTeamEncoded'] = fixtures['AwayTeam'].apply(lambda x: team_encoder.transform([x])[0] if x in team_encoder.classes_ else -1)

fixtures['PredictedHomeGoals'] = home_model.predict(fixtures[['HomeTeamEncoded', 'AwayTeamEncoded']])
fixtures['PredictedAwayGoals'] = away_model.predict(fixtures[['HomeTeamEncoded', 'AwayTeamEncoded']])

fixtures['Winner'] = np.where(
    fixtures['PredictedHomeGoals'] > fixtures['PredictedAwayGoals'], fixtures['HomeTeam'],
    np.where(fixtures['PredictedHomeGoals'] < fixtures['PredictedAwayGoals'], fixtures['AwayTeam'], 'Draw')
)


In [6]:
fixtures.to_csv("fifa_worldcup_2026_predictions.csv", index=False)
standings = {}


In [8]:

group_stage_results = pd.read_csv("fifa_worldcup_2026_predictions.csv")
print(group_stage_results.head())  # Debugging step to confirm data is loaded

for _, row in group_stage_results.iterrows():
    home, away, winner = row['HomeTeam'], row['AwayTeam'], row['Winner']
    standings.setdefault(home, {'Points': 0, 'GD': 0})
    standings.setdefault(away, {'Points': 0, 'GD': 0})
    
    if winner == home:
        standings[home]['Points'] += 3
    elif winner == away:
        standings[away]['Points'] += 3
    else:
        standings[home]['Points'] += 1
        standings[away]['Points'] += 1

standings_df = pd.DataFrame.from_dict(standings, orient='index').reset_index().rename(columns={'index': 'Team'})
standings_df = standings_df.sort_values(by=['Points', 'GD'], ascending=False)


     HomeTeam     score    AwayTeam  year  HomeTeamEncoded  AwayTeamEncoded  \
0         USA   Match 1      Mexico  2026               -1               40   
1      Canada   Match 2  Costa Rica  2026               11               15   
2         USA  Match 18      Canada  2026               -1               11   
3      Mexico  Match 19  Costa Rica  2026               40               15   
4  Costa Rica  Match 35      Canada  2026               15               11   

   PredictedHomeGoals  PredictedAwayGoals      Winner  
0            1.507214            1.227901         USA  
1            0.786000            1.715000  Costa Rica  
2            0.377500            0.972000      Canada  
3            1.173714            1.121476      Mexico  
4            2.525000            1.225000  Costa Rica  


In [9]:
standings_df.to_csv("fifa_worldcup_2026_standings.csv", index=False)


In [39]:
standings_df = pd.read_csv("fifa_worldcup_2026_standings.csv")
qualified_teams = standings_df.head(16)['Team'].tolist()

print(f"\n🏆 Qualified Teams for Knockout Stage: {qualified_teams}")

# Load historical match data
df = pd.read_csv("clean_fifa_worldcup_matches.csv").dropna()

# Encode teams correctly
team_encoder = LabelEncoder()
all_teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()  # Get all unique teams
team_encoder.fit(all_teams)  # Fit the encoder on all teams

# Transform both HomeTeam and AwayTeam using the same encoder
df['HomeTeamEncoded'] = team_encoder.transform(df['HomeTeam'])
df['AwayTeamEncoded'] = team_encoder.transform(df['AwayTeam'])

# Prepare training data
X = df[['HomeTeamEncoded', 'AwayTeamEncoded']]
y_home = df['HomeGoals']
y_away = df['AwayGoals']

# Train-test split
X_train, X_test, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.2, random_state=42)
X_train, X_test, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.2, random_state=42)

# Train the models
home_model = RandomForestRegressor(n_estimators=100, random_state=42)
away_model = RandomForestRegressor(n_estimators=100, random_state=42)
home_model.fit(X_train, y_train_home)
away_model.fit(X_train, y_train_away)

# Function to encode teams safely
def encode_team(team):
    if team in team_encoder.classes_:
        return team_encoder.transform([team])[0]
    else:
        return -1 


🏆 Qualified Teams for Knockout Stage: ['Mexico', 'Costa Rica', 'Brazil', 'France', 'Spain', 'Portugal', 'Netherlands', 'Italy', 'USA', 'Canada', 'Argentina', 'Germany', 'Japan', 'England', 'Denmark', 'Belgium']


In [35]:
print(X_train.shape, y_train_home.shape, y_train_away.shape)


(771, 2) (771,) (771,)


In [44]:
knockout_rounds = ["Round of 16", "Quarterfinals", "Semifinals", "Final"]
remaining_teams = qualified_teams.copy()

# Store match results
tournament_results = []
final_standings = []

for round_name in knockout_rounds:
    print(f"\n🔹 {round_name} Matches:")

    if len(remaining_teams) == 1:  # If only one team remains, declare winner
        break

    next_round_teams = []

    # Handle odd number of teams (Bye system)
    if len(remaining_teams) % 2 == 1:
        bye_team = remaining_teams.pop()  # Remove last team
        print(f"⚠️ {bye_team} gets a bye and advances to the next round.")
        next_round_teams.append(bye_team)
        final_standings.append((bye_team, round_name))  # Track tournament position

    for i in range(0, len(remaining_teams), 2):
        team1, team2 = remaining_teams[i], remaining_teams[i + 1]
        encoded_team1, encoded_team2 = encode_team(team1), encode_team(team2)

        if encoded_team1 == -1 or encoded_team2 == -1:
            print(f"⚠️ Error: Team encoding issue with {team1} or {team2}")
            continue

        # Predict match result
        match_features = pd.DataFrame([[encoded_team1, encoded_team2]], columns=['HomeTeamEncoded', 'AwayTeamEncoded'])
        predicted_home_goals = home_model.predict(match_features)[0]
        predicted_away_goals = away_model.predict(match_features)[0]

        # Determine winner
        if predicted_home_goals > predicted_away_goals:
            winner, loser = team1, team2
        elif predicted_home_goals < predicted_away_goals:
            winner, loser = team2, team1
        else:
            winner = np.random.choice([team1, team2])  # Random choice on draw
            loser = team1 if winner == team2 else team2

        print(f"⚽ {team1} ({predicted_home_goals:.1f}) vs {team2} ({predicted_away_goals:.1f}) → Winner: {winner}")
        next_round_teams.append(winner)
        
        # Store results
        tournament_results.append([round_name, team1, team2, predicted_home_goals, predicted_away_goals, winner])
        final_standings.append((loser, round_name))  # Track when a team was eliminated

    remaining_teams = next_round_teams  # Advance to next round
    print(f"✅ Advancing Teams: {remaining_teams}")

# Declare final winner
final_winner = remaining_teams[0]
print(f"\n🏆🏆🏆 FIFA World Cup 2026 Champion: {final_winner} 🏆🏆🏆")
final_standings.append((final_winner, "Champion"))

# Convert to DataFrame
results_df = pd.DataFrame(tournament_results, columns=["Round", "Team1", "Team2", "Predicted Goals Team1", "Predicted Goals Team2", "Winner"])
standings_df = pd.DataFrame(final_standings, columns=["Team", "Position"])

# Export CSV files
results_df.to_csv("predicted_tournament_results.csv", index=False)
standings_df.to_csv("final_tournament_standings.csv", index=False)

# Update previous standings CSV
previous_standings = pd.read_csv("fifa_worldcup_2026_standings.csv")
updated_standings = pd.concat([previous_standings, standings_df], ignore_index=True)
updated_standings.to_csv("fifa_worldcup_2026_standings_Final_Updated.csv", index=False)



🔹 Round of 16 Matches:
⚽ Mexico (1.2) vs Costa Rica (1.0) → Winner: Mexico
⚽ Brazil (1.6) vs France (1.8) → Winner: France
⚽ Spain (1.0) vs Portugal (0.5) → Winner: Spain
⚽ Netherlands (1.4) vs Italy (0.4) → Winner: Netherlands
⚠️ Error: Team encoding issue with USA or Canada
⚽ Argentina (0.9) vs Germany (3.8) → Winner: Germany
⚽ Japan (1.3) vs England (0.6) → Winner: Japan
⚽ Denmark (3.1) vs Belgium (1.8) → Winner: Denmark
✅ Advancing Teams: ['Mexico', 'France', 'Spain', 'Netherlands', 'Germany', 'Japan', 'Denmark']

🔹 Quarterfinals Matches:
⚠️ Denmark gets a bye and advances to the next round.
⚽ Mexico (2.2) vs France (0.8) → Winner: Mexico
⚽ Spain (1.3) vs Netherlands (4.1) → Winner: Netherlands
⚽ Germany (1.6) vs Japan (1.3) → Winner: Germany
✅ Advancing Teams: ['Denmark', 'Mexico', 'Netherlands', 'Germany']

🔹 Semifinals Matches:
⚽ Denmark (1.0) vs Mexico (2.0) → Winner: Mexico
⚽ Netherlands (1.6) vs Germany (0.9) → Winner: Netherlands
✅ Advancing Teams: ['Mexico', 'Netherlands']